In [1]:
import numpy as np
import os

In [5]:
def readPzs():
    #This fn reads in the puzzles from 'testExamples.xsb' - default or
    #the given file name.
    fname=input('File name:')
    if (fname==''):
        fname='testExamples.xsb'
        print('Using file ',fname)
    os.chdir("C:/Users/kollu/OneDrive/Documents/SokAI")#change this if you want to use it
    try: 
        with open(fname) as rd:
            #each puzzle ends with an empty line or eof
            #a puzzle may be preceded by comment lines containing ; or new lines
            #pzs=dictionary of puzzles, pz=individual puzzle as list of lines where
            #each line is a string, pzno=puzzle no.
            pzno,pzs,pz=0,dict(),list()
            mlen,lines=0,0#mlen=max len of line in pz, lines=no. of lines in pz

            for line in rd:
                if (line==''):#end of file
                    if (pz!=[]):#eof ends puzzle
                        pzno+=1
                        pzs[pzno]=(pz,mlen,lines)
                    break;
                elif (';' in line):#comment, skip line 
                    if (pz!=[]):#pz!=[] the puzzle ends not with newline but with comment
                        pzno+=1
                        pzs[pzno]=(pz,mlen,lines)
                        pz,mlen,lines=list(),0,0
                    continue
                elif (line=='\n'):#either end of pz or empty comment line
                    if (pz==[]):#empty comment line, skip
                        continue
                    else: #pz has ended, add to pzs, with len, lines
                        pzno+=1
                        pzs[pzno]=(pz,mlen,lines)#add (pz,mlen) to dictionary
                        pz,mlen,lines=list(),0,0#reset pz, mlen, lines for next puzzle
                        continue
                else:#start or next line of a puzzle
                    if (pz==[]):#possible puzzle start
                        if ('#' not in line):#possible error. Boundary should contain '#'
                            print("Possible error in puzzle:", line)
                            exit()
                    #next line of puzzle    
                    mlen=max(mlen,len(line)-1)
                    lines+=1
                    pz.append(line.rstrip('\n'))
            if (pz!=[]):#puzzle ended with eof not newline
                pzno+=1
                pzs[pzno]=(pz,mlen,lines)
    except Exception as ex:
        print('Error:', ex)
    else:
        return pzs

In [13]:
#This cell will store all puzzles in the dictionary pzs and display them.
#Run this cell after running the cells above.
pzs=readPzs()#read in the puzzles
pzs

File name:sokAI
Error: [Errno 2] No such file or directory: 'sokAI'


In [7]:
#Converts given puzzle number to numpy array with the following code for the symbols:
#space=0, #=1, $=2, .=3, @=4, if box in goal position 5.
#Final puzzle representation is a numpy array.
#State is modelled by: agent position, lists of box positions and goal positions (fixed).
def pzToArray(pzs, pzno):#pzno is the puzzle no from the pzs puzzle dictionary
    #returns tuple of (puzzle,) agent row, agent col, goals, boxes).
    goals,boxes=list(),list()#goal, box positions
    pz,mlen,lines=pzs[pzno]
    pza=np.zeros((lines,mlen),dtype=np.uint8)#initialize ndarray
    row=0
    for line in pz:
        col=0
        for c in line:
            if (c=='#'): pza[row][col]=1
            elif (c=='$'): 
                pza[row][col]=2
                boxes.append((row,col))
            elif (c=='.'): 
                pza[row][col]=3
                goals.append((row,col))
            elif (c=='@'): 
                pza[row][col]=4
                ar,ac=row,col#agent's row, col
            elif (c==' '): pass
            else: 
                print('Illegal char in puzzle ',c,'in puzzle. Line = ',row+1)
                exit()
            col+=1
        row+=1
    return (pza, ar, ac, goals, boxes)           

In [8]:
def raiseExc(msg):
    raise ValueError(msg)

def agentMove(c,pza,ar,ac,boxes):
    #Illegal move if agent moves to a square that is a wall, another agent or outside puzzle
    col=(ac+1) if (c=='r') else (ac-1) if (c=='l') else ac
    row=(ar+1) if (c=='d') else (ar-1) if (c=='u') else ar
    if (col<0 or row<0 or pza[row][col]==1 or (row,col) in boxes): 
        return None#illegal move
    else: return (row,col)
    
def pushMove(c,pza,ar,ac,boxes):
    col=(ac+1) if (c=='R') else (ac-1) if (c=='L') else ac
    row=(ar+1) if (c=='D') else (ar-1) if (c=='U') else ar
    if ((row,col) in boxes):#agent adjacent to box
        #Calc. new position of pushed box
        bcol=(col+1) if (c=='R') else (col-1) if (c=='L') else col
        brow=(row+1) if (c=='D') else (row-1) if (c=='U') else row
        if (bcol<0 or brow<0 or pza[brow][bcol]==1 or (brow,bcol) in boxes): 
            return None#illegal push move
        else:
            boxes[boxes.index((row,col))]=(brow,bcol)
            return (row,col,boxes)
    else: return None

def simSoln(pzs, pzno, soln):#simulate the soln.
    #soln=string of chars from 'lurdLURD'
    pza,ar,ac,goals,boxes=pzToArray(pzs,pzno)#puzzle array and initial state
    for c in soln:
        if (c in 'lurd'):#only agent moves
            apos=agentMove(c,pza,ar,ac,boxes)#returns new agent pos or None
            if (apos is None): raiseExc('Illegal agent move '+c+'. Agent position '+str((ar,ac)))
            else:#legal agent move, update agent pos
                ar,ac=apos
        elif (c in 'LURD'):#push move
            newState=pushMove(c,pza,ar,ac,boxes)#returns new state or None
            if (newState is None): raiseExc('Illegal push move '+c+'. Agent position '+str((ar,ac)))
            else:#legal push move, update state
                ar,ac,boxes=newState
        else:#Illegal move character
            raiseExc('Illegal move character '+c)
    #simulation done check if all boxes in goal positions
    for p in boxes:
        if (p in goals):
            pza[p[0]][p[1]]=5#box in goal position coded by 5          
            goals.remove(p)#remove box position from goals
            
    if (goals==[]):
        print('Puzzle solution was correct. Final state:\n')
    else:
        print('Puzzle solution was incorrect. Final state:\n')
    print(pza)#print the final state

In [11]:
#Driver program for the simulation. 
#Run this cell after running all the cells above to check your solutions to the 7 puzzles in file t1.xsb.

file=input('Solution file name = ')
if (file==''): file='tst'#tst is the default solution file name
pzno=int(input('Give puzzle no. (1 to 7) = '))# I added a 7th puzzle variant of puzzle 1
os.chdir("C:/Users/kollu/OneDrive/Documents/SokAI")#change this if you want to use this code
try:
    with open(file) as srd:
        soln=srd.readline().rstrip('\n')
        print(soln)
        simSoln(pzs,pzno,soln)
except Exception as ex:
    print('Error: ',ex)

Solution file name = output.xsb
Give puzzle no. (1 to 7) = 5

Error:  5
